# DATA 608 Assignment 4
# Dan Smilowitz

## Data Preparation

In [1]:
import pandas as pd
# read in data
river = pd.read_csv('https://raw.githubusercontent.com/jlaurito/CUNY_IS608/master/lecture4/data/riverkeeper_data_2013.csv')

In [2]:
river.dtypes

Site                 object
Date                 object
EnteroCount          object
FourDayRainTotal    float64
SampleCount           int64
dtype: object

In [3]:
# covert Date and EnteroCount to proper data types
river['Date'] = pd.to_datetime(river['Date'], format='%m/%d/%Y')
river['EnteroCount'] = pd.to_numeric(river['EnteroCount'], errors='coerce') # this handles extra < or > characters
# remove NAs
river = river.dropna()
river.head()

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount
0,Hudson above Mohawk River,2011-10-16,1733.0,1.5,35
1,Hudson above Mohawk River,2013-10-21,4.0,0.2,35
2,Hudson above Mohawk River,2013-09-21,20.0,0.0,35
3,Hudson above Mohawk River,2013-08-19,6.0,0.0,35
4,Hudson above Mohawk River,2013-07-21,31.0,0.0,35


## Questions
### Create lists & graphs of the best and worst places to swim in the dataset.

In [4]:
# calculate average health of sites
q1 = river.groupby('Site', as_index=False).mean()[['Site', 'EnteroCount']].round(2)
# return best and worst N sites
N = 10
best = q1.sort_values(by = 'EnteroCount', ascending=True).head(N)
worst = q1.sort_values(by = 'EnteroCount', ascending=False).head(N)

In [5]:
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.models.layouts import WidgetBox
from bokeh.layouts import row, column, layout
from bokeh.models.ranges import FactorRange
from bokeh.charts import Bar, output_notebook, show
from bokeh.charts.attributes import CatAttr

# output to notebook
output_notebook()

# make tables
source_worst = ColumnDataSource(worst)
source_best = ColumnDataSource(best)

columns = [TableColumn(field='Site', title='Site'), TableColumn(field='EnteroCount', title='Avg Count')]

table_best = DataTable(source=source_best, columns=columns, row_headers=False)
table_worst = DataTable(source=source_worst, columns=columns, row_headers=False)

# make bar charts
bar_best = Bar(best, label=CatAttr(df=worst, columns='Site', sort=False), values='EnteroCount', color='limegreen',
               title='Best Places to Swim', legend=None, ylabel='Mean Enterococcus Count', xlabel='')
bar_worst = Bar(worst, label=CatAttr(df=worst, columns='Site', sort=False), values='EnteroCount', color='crimson',
                title='Worst Places to Swim', legend=None, ylabel='Mean Enterococcus Count', xlabel='')

# show tables & charts
best_worst = layout([
        [bar_best, table_best],
        [bar_worst, table_worst]
    ], sizing_mode='scale_width')
show(best_worst)

Loading BokehJS ...

### Which sites have been tested most regularly? Which ones have long gaps between tests?

In [6]:
# find most frequently sampled sites
q2 = river.groupby('Site', as_index=False).max()[['Site', 'SampleCount']]
most = q2.sort_values(by = 'SampleCount', ascending=False).head(N/2)
least = q2.sort_values(by = 'SampleCount', ascending=True).head(N/2)

# create table
source_most = ColumnDataSource(most)
source_least = ColumnDataSource(least)

columns = [TableColumn(field='Site', title='Site'), TableColumn(field='SampleCount', title='# Samples')]

table_most = DataTable(source=source_most, columns=columns, row_headers=False, fit_columns=True)
table_least = DataTable(source=source_least, columns=columns, row_headers=False, fit_columns=True)

# create bar charts
bar_most = Bar(most, label=CatAttr(df=most, columns='Site', sort=False), values='SampleCount', color='limegreen',
               title='Most-Sampled Sites', legend=None, ylabel='Number of Samples', xlabel='')
bar_least = Bar(least, label=CatAttr(df=least, columns='Site', sort=False), values='SampleCount', color='crimson',
                title='Least-Sampled Sites', legend=None, ylabel='Number of Samples', xlabel='')

# show tables and charts
most_least = layout([
        [bar_most, table_most],
        [bar_least, table_least]
    ], sizing_mode='scale_width')
show(most_least)

### Pick out 5-10 sites and visually compare how regularly their water quality is tested.

In [7]:
from bokeh.plotting import figure

# select 5 best and 5 worst cities to swim
sites_freq = worst['Site'].iloc[:5].tolist() + best['Site'].iloc[:5].tolist()[::-1]
test_freq = river[river['Site'].isin(sites_freq)][['Site', 'Date']]

# create categorical vs. timeseries plot
freq_plot = figure(title='Frequency of Water Quality Tests for Best & Worst Places to Swim',
                   x_axis_type='datetime', width=900, y_range=sites_freq)
freq_plot.circle(test_freq['Date'], test_freq['Site'], size=8, alpha=0.5, line_color=None)

show(freq_plot)

### Is there a relationship between the amount of rain and water quality?  Show this relationship graphically.

In [8]:
from bokeh.models import Label
from scipy import stats

# create scatterplot with hover
rain_quality = figure(title='Count vs. Rain', width=900)
rain_quality.circle(river['FourDayRainTotal'], river['EnteroCount'], 
                    fill_alpha = 0.2, line_color=None, size=8)

# fit & plot line 
model = stats.linregress(river['FourDayRainTotal'], river['EnteroCount'])
fit = model.intercept + model.slope * river['FourDayRainTotal']
rain_quality.line(river['FourDayRainTotal'], fit, line_color='red', 
                  line_dash='dotted', line_width=2)

# label fit line
lin_eq = Label(x=6.5, y=2500, text='y = %.2f + %.2f * x' 
               %(model.intercept, model.slope), text_color='red')
rain_quality.add_layout(lin_eq)
lin_r2 = Label(x=6.5, y=1500, text='R^2 = %.4f' 
               % model.rvalue**2, text_color='red')
rain_quality.add_layout(lin_r2)

show(rain_quality)

 ### If you can, estimate the effect of rain on quality at different sites and create a visualization to compare them.

In [9]:
# select best two and worst two sites
rain_sites = [sites_freq[i] for i in [0,1,8,9]]

# create scatterplots for each of four selected sites
rq_plots = []
for i in range(4):
    data = river[river['Site'] == rain_sites[i]]
    p = figure(title=sites_freq[i])
    p.circle(data['FourDayRainTotal'], river['EnteroCount'], 
             fill_alpha = 0.5, line_color=None, size=8)
    rq_plots.append(p)

# show plots    
site_rq = layout([
        [rq_plots[0], rq_plots[1]],
        [rq_plots[2], rq_plots[3]]
    ], sizing_mode='scale_width')
show(site_rq)